In [ ]:
import os
import cv2
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 사용자 정의 라이브러리
from custom_dataset import XRayDataset, CLASSES 
from SAM2UNet import SAM2UNet

# 시드 고정 (선택 사항)
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_seed(42)
print("Libraries loaded & Seed set.")

In [ ]:
# 1. 데이터 경로
ROOT_IMG = "../data/train/DCM" 
ROOT_LBL = "../data/train/outputs_json"

# 2. 모델 체크포인트 경로
MODEL_PATH = "../sam2_unet_result_checkpoints/experiment2.pth"
HIERA_PATH = "../checkpoints/sam2_hiera_large.pt"

# 3. 디바이스 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 4. 데이터셋 변환 (Validation/Test용)
# 학습 때 사용한 크기와 맞춰줘야 함
IMG_SIZE = 1024 
valid_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    ToTensorV2()
])

print(f"Device: {DEVICE}")
print(f"Image Size: {IMG_SIZE}")

In [ ]:
def generate_class_wise_overlap_target(target):
    """
    각 클래스별로 '다른 뼈와 겹치는 영역'만 남기고 나머지는 0으로 만드는 함수
    Args:
        target: [Batch, 29, H, W] (Binary Mask)
    Returns:
        class_wise_overlap: 겹치는 부위만 남은 클래스별 마스크
        global_overlap_mask: 전체 이미지에서 겹침이 발생한 모든 영역
    """
    with torch.no_grad():
        # 1. 픽셀별로 뼈가 몇 개 있는지 계산
        count_map = torch.sum(target, dim=1, keepdim=True)
        # 2. 뼈가 2개 이상 있는 곳 찾기 (Overlap)
        global_overlap_mask = (count_map >= 2.0).float()
        # 3. 각 클래스 마스크에 Overlap 마스크 적용
        class_wise_overlap = target * global_overlap_mask
    return class_wise_overlap, global_overlap_mask

def generate_masked_edge_target(target):
    """
    겹치는 영역 내부에 있는 경계선(Edge)만 추출
    """
    with torch.no_grad():
        dilated = F.max_pool2d(target, kernel_size=3, stride=1, padding=1)
        eroded = -F.max_pool2d(-target, kernel_size=3, stride=1, padding=1)
        raw_edges = dilated - eroded
        
        count_map = torch.sum(dilated, dim=1, keepdim=True)
        overlap_mask = (count_map >= 2.0).float()
        
        masked_edges = raw_edges * overlap_mask
    return masked_edges

def generate_inner_edge_target(target):
    """
    뼈의 안쪽으로 파고드는(침식 기반) 내부 경계 생성
    """
    with torch.no_grad():
        # kernel=5로 조금 더 깊게 침식
        eroded = -F.max_pool2d(-target, kernel_size=5, stride=1, padding=2)
        inner_edges = target - eroded
        
        count_map = torch.sum(target, dim=1, keepdim=True)
        overlap_mask = (count_map >= 2.0).float()
        
        masked_edges = inner_edges * overlap_mask
    return masked_edges, overlap_mask

In [ ]:
def visualize_3_channels(dataset, index=0):
    """
    Dataset의 __getitem__이 반환하는 3개 채널(예: 원본, Canny, Laplacian 등)을 시각화
    """
    data = dataset[index]
    image_tensor = data['image'] # (3, H, W)
    image_np = image_tensor.permute(1, 2, 0).numpy()
    
    plt.figure(figsize=(20, 5))
    titles = ["Ch1: Original", "Ch2: Contrast/Edge", "Ch3: Boundary/Laplacian"]
    
    for i in range(3):
        plt.subplot(1, 4, i+1)
        plt.imshow(image_np[:, :, i], cmap='gray')
        plt.title(titles[i])
        plt.axis('off')
        
    plt.subplot(1, 4, 4)
    plt.imshow(image_np)
    plt.title("Merged (RGB Model View)")
    plt.axis('off')
    plt.show()

def analyze_clahe_difference(dataset, index=0):
    """
    CLAHE Clip Limit에 따른 이미지 변화량(Difference)을 히트맵으로 분석 함수
    """
    data = dataset[index]
    image_np = data['image'].permute(1, 2, 0).numpy()
    
    # 데이터셋 구성에 따라 인덱스가 다를 수 있음 (여기서는 예시)
    orig = image_np[:, :, 0]
    processed_1 = image_np[:, :, 1]
    
    diff = np.abs(processed_1 - orig)
    
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 3, 1); plt.imshow(orig, cmap='gray'); plt.title("Original")
    plt.subplot(1, 3, 2); plt.imshow(processed_1, cmap='gray'); plt.title("Processed (CLAHE/Edge)")
    
    plt.subplot(1, 3, 3)
    plt.imshow(diff, cmap='magma')
    plt.colorbar()
    plt.title("Difference Heatmap")
    plt.show()

In [ ]:
def visualize_all_classes_metric(model, dataset, index=0, device=DEVICE):
    """
    특정 이미지에 대해 29개 클래스별 Dice Score, TP, FP, FN 픽셀 수를 계산하여 출력
    """
    data = dataset[index]
    image_tensor = data['image'].unsqueeze(0).to(device)
    mask_tensor = data['label'].permute(2, 0, 1).unsqueeze(0).float().to(device)
    
    model.eval()
    with torch.no_grad():
        # 모델의 리턴값 구조에 따라 수정 필요 (여기서는 4번째가 edge라고 가정)
        output = model(image_tensor)
        # SAM2UNet 리턴이 (mask, ds1, ds2, edge) 형태라면:
        if isinstance(output, tuple):
             _, _, _, pred_edge = output
        else:
            pred_edge = output

        # 타겟 생성 (Overlap 기준)
        target_edge, _ = generate_class_wise_overlap_target(mask_tensor) 
    
    # Thresholding
    pred_edge_binary = (torch.sigmoid(pred_edge) > 0.5).float()
    
    target_np = target_edge.squeeze(0).cpu().numpy()
    pred_np = pred_edge_binary.squeeze(0).cpu().numpy()
    
    print(f"--- [Image Index {index}] Class-wise Report ---")
    print(f"{'ID':<4} {'Class Name':<15} {'Dice':<8} {'TP':<6} {'FP':<6} {'FN':<6} {'Status'}")
    
    for idx, class_name in enumerate(CLASSES):
        gt = target_np[idx] > 0.5
        pred = pred_np[idx] > 0.5
        
        tp = np.sum(gt & pred)
        fp = np.sum((~gt) & pred)
        fn = np.sum(gt & (~pred))
        
        dice = (2 * tp) / (2 * tp + fp + fn + 1e-5)
        
        status = ""
        if np.sum(gt) == 0 and np.sum(pred) > 0: status = "⚠️ FP Warning"
        if np.sum(gt) == 0 and np.sum(pred) == 0: status = "Clean"
        
        print(f"{idx+1:02d}   {class_name:<15} {dice:.4f}   {tp:<6} {fp:<6} {fn:<6} {status}")

In [ ]:
def visualize_strict_combined(model, dataset, index, device=DEVICE):
    """
    모델의 예측 에러를 시각화
    Red: 놓친 부분 (Miss/FN), Blue: 잘못 예측한 노이즈 (Noise/FP)
    """
    model.eval()
    data = dataset[index]
    image_tensor = data['image'].unsqueeze(0).to(device)
    target_origin = data['label'].permute(2, 0, 1).to(device) # (29, H, W)
    
    with torch.no_grad():
        output = model(image_tensor)
        # 모델 출력 구조에 맞춰 로짓 선택 (여기서는 첫번째가 main mask라고 가정)
        if isinstance(output, tuple): pred_logits = output[0]
        else: pred_logits = output
            
        pred_probs = torch.sigmoid(pred_logits)
        
    pred_mask = (pred_probs[0] > 0.5).cpu().numpy()
    gt_mask = (target_origin > 0).cpu().numpy()
    image_np = data['image'].permute(1, 2, 0).cpu().numpy()
    
    # 에러 누적
    total_miss = np.zeros(gt_mask.shape[1:], dtype=bool)
    total_noise = np.zeros(gt_mask.shape[1:], dtype=bool)
    
    for c in range(29):
        p, g = pred_mask[c], gt_mask[c]
        total_miss |= (g & ~p) # FN
        total_noise |= (~g & p) # FP

    # 시각화용 오버레이
    vis_miss = np.zeros((*total_miss.shape, 4)); vis_miss[total_miss] = [1, 0, 0, 0.6]
    vis_noise = np.zeros((*total_noise.shape, 4)); vis_noise[total_noise] = [0, 0, 1, 0.6]

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1); plt.imshow(image_np); plt.title("Original")
    plt.subplot(1, 3, 2); plt.imshow(image_np); plt.imshow(vis_miss); plt.title("Miss (Red: FN)")
    plt.subplot(1, 3, 3); plt.imshow(image_np); plt.imshow(vis_noise); plt.title("Noise (Blue: FP)")
    plt.show()

In [ ]:
def visualize_strict_combined(model, dataset, index, device=DEVICE):
    """
    모델의 예측 에러를 시각화
    Red: 놓친 부분 (Miss/FN), Blue: 잘못 예측한 노이즈 (Noise/FP)
    """
    model.eval()
    data = dataset[index]
    image_tensor = data['image'].unsqueeze(0).to(device)
    target_origin = data['label'].permute(2, 0, 1).to(device) # (29, H, W)
    
    with torch.no_grad():
        output = model(image_tensor)
        # 모델 출력 구조에 맞춰 로짓 선택 (여기서는 첫번째가 main mask라고 가정)
        if isinstance(output, tuple): pred_logits = output[0]
        else: pred_logits = output
            
        pred_probs = torch.sigmoid(pred_logits)
        
    pred_mask = (pred_probs[0] > 0.5).cpu().numpy()
    gt_mask = (target_origin > 0).cpu().numpy()
    image_np = data['image'].permute(1, 2, 0).cpu().numpy()
    
    # 에러 누적
    total_miss = np.zeros(gt_mask.shape[1:], dtype=bool)
    total_noise = np.zeros(gt_mask.shape[1:], dtype=bool)
    
    for c in range(29):
        p, g = pred_mask[c], gt_mask[c]
        total_miss |= (g & ~p) # FN
        total_noise |= (~g & p) # FP

    # 시각화용 오버레이
    vis_miss = np.zeros((*total_miss.shape, 4)); vis_miss[total_miss] = [1, 0, 0, 0.6]
    vis_noise = np.zeros((*total_noise.shape, 4)); vis_noise[total_noise] = [0, 0, 1, 0.6]

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1); plt.imshow(image_np); plt.title("Original")
    plt.subplot(1, 3, 2); plt.imshow(image_np); plt.imshow(vis_miss); plt.title("Miss (Red: FN)")
    plt.subplot(1, 3, 3); plt.imshow(image_np); plt.imshow(vis_noise); plt.title("Noise (Blue: FP)")
    plt.show()

In [ ]:
if __name__ == "__main__":
    if os.path.exists(ROOT_IMG) and os.path.exists(MODEL_PATH):
        # 1. 데이터셋 및 모델 로드
        dataset = XRayDataset(ROOT_IMG, ROOT_LBL, is_train=False, transforms=valid_transform)
        
        model = SAM2UNet(checkpoint_path=HIERA_PATH)
        checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
        
        # state_dict 키 처리 (저장 방식에 따라 다를 수 있음)
        if 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
        else:
            model.load_state_dict(checkpoint)
            
        model.to(DEVICE)
        print("모델 및 데이터셋 로드 완료")
        
        # [A] 입력 데이터 확인 (채널별 시각화)
        # visualize_3_channels(dataset, index=30)
        
        # [B] CLAHE 차이 분석
        # analyze_clahe_difference(dataset, index=30)
        
        # [C] 전체 클래스 메트릭(Dice/TP/FP/FN) 리포트 출력
        visualize_all_classes_metric(model, dataset, index=30, device=DEVICE)
        
        # [D] 엄격한 에러 시각화 (Miss vs Noise)
        # visualize_strict_combined(model, dataset, index=30, device=DEVICE)
        
        # [E] 특정 뼈(예: Pisiform)의 겹침 영역 예측 성능 시각화
        # visualize_class_wise_overlap_prediction(model, dataset, index=30, target_bones=["Pisiform", "Lunate"], device=DEVICE)

    else:
        print("경로 오류: 데이터셋 폴더나 모델 파일을 찾을 수 없음")